In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium import FeatureGroup
from config import vworld_key
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# 폴리곤을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [3]:
# 라인스트링을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [4]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]
crp_df

,type,properties.fid,properties.구역명,geometry.type,geometry.coordinates,geometry,centroid_polygon_geometry
0,Feature,5,도시재생구역도,Polygon,"[[[127.4833417, 36.6318644], [127.4833417, 36....","POLYGON ((127.4833417 36.6318644, 127.4833417 ...",POLYGON ((127.50347965416363 36.63603755594320...


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.60720970472786, 127.63641867895493], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in crp_df.iterrows():
    popup_text = f"{row['properties.구역명']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='blue': {'fillColor': 'blue', 'color': 'black', 'weight': 0.1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))  # 팝업 메시지 추가
    
m

### 격자(매핑용)

In [7]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df

,type,properties.gid,geometry.type,geometry.coordinates,geometry
0,Feature,라바122456,Polygon,"[[[127.63641867895493, 36.608165744671886], [1...",POLYGON ((127.63641867895493 36.60816574467188...
1,Feature,다바841455,Polygon,"[[[127.32221067151384, 36.60720970472786], [12...",POLYGON ((127.32221067151384 36.60720970472786...
2,Feature,라바189471,Polygon,"[[[127.71137365974974, 36.62157886605511], [12...",POLYGON ((127.71137365974974 36.62157886605511...
3,Feature,다바852482,Polygon,"[[[127.33445848271631, 36.631567839444976], [1...",POLYGON ((127.33445848271631 36.63156783944497...
4,Feature,다바959488,Polygon,"[[[127.45413724007568, 36.63708300728262], [12...",POLYGON ((127.45413724007568 36.63708300728262...
...,...,...,...,...,...
95251,Feature,다바868478,Polygon,"[[[127.35236168440159, 36.62798537617458], [12...",POLYGON ((127.35236168440159 36.62798537617458...
95252,Feature,다바974324,Polygon,"[[[127.47097166337947, 36.48924077421588], [12...",POLYGON ((127.47097166337947 36.48924077421588...
95253,Feature,라바151492,Polygon,"[[[127.66891675313607, 36.640578073348856], [1...",POLYGON ((127.66891675313607 36.64057807334885...
95254,Feature,다바841537,Polygon,"[[[127.3220406982908, 36.681132105366395], [12...",POLYGON ((127.3220406982908 36.681132105366395...


In [8]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].within(polygon)].reset_index(drop=True)
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].within(polygon_expanded)].reset_index(drop=True)

In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.60720970472786, 127.63641867895493], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_grid_expanded.iterrows():
    popup_text = f"{row['properties.gid']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'fillColor': 'black', 'color': 'black', 'weight': 0.1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))  # 팝업 메시지 추가
    
m

### 청주시 도시재생구역 거주인구 필터링 및 격자에 매핑

In [10]:
respop_df = pd.read_csv('SBJ_2309_001/1.청주시_거주인구.csv')
#원도심 영역 grid id 리스트 로드 및 respop_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_respop = respop_df[respop_df['gid'].isin(grid_id)]
filtered_respop = filtered_respop.fillna(0)
filtered_respop = pd.concat([filtered_respop.iloc[:, :2], filtered_respop.iloc[:, 2:].astype(int)], axis=1)
filtered_respop.rename(columns = {"year": "연도"}, inplace = True)
# 성별구분 없이 통합 및 세대별 인구수 데이터프레임으로 변환
for i in range(2, len(filtered_respop.columns), 2):
    col_name = f'{filtered_respop.columns[i][2:4]}대' if filtered_respop.columns[i][2] != '1' else f'{filtered_respop.columns[i][2:5]}대'
    filtered_respop[col_name] = filtered_respop.iloc[:, i:i+2].sum(axis=1)


filtered_respop = filtered_respop[list(filtered_respop.columns[:2]) + list(filtered_respop.columns[20:])]
filtered_respop['종합'] = filtered_respop.iloc[:, 2:].sum(axis=1)
filtered_respop =  filtered_respop[filtered_respop['종합'] != 0] # 거주인구가 없는 격자 필터링(제외)
filtered_respop['고령인구_비율'] = filtered_respop.iloc[:, 6:-1].sum(axis=1) / filtered_respop.iloc[:, -1]
filtered_respop = filtered_respop.fillna(0)
filtered_respop['고령인구_비율'] = filtered_respop['고령인구_비율'].apply(lambda x : round(x*100, 2))
filtered_respop

,gid,연도,20대,30대,40대,50대,60대,70대,80대,90대,100대,종합,고령인구_비율
685,다바991491,2021,10,8,6,14,12,0,0,0,0,50,24.00
971,다바990495,2021,81,118,70,67,33,6,10,0,0,385,12.73
1360,다바989489,2021,81,31,17,18,14,0,0,0,0,161,8.70
1655,다바990491,2021,0,0,6,0,0,0,0,0,0,6,0.00
1662,다바990479,2021,6,8,0,0,20,6,0,0,0,40,65.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
393078,다바990489,2022,14,0,7,23,18,20,0,0,0,82,46.34
393250,다바992483,2022,212,177,168,240,126,83,41,6,0,1053,24.31
393304,다바986481,2022,6,0,0,9,20,0,0,0,0,35,57.14
395475,다바991495,2022,10,0,0,15,0,0,0,0,0,25,0.00


국제연합(UN)의 기준에 따르면 전체 인구에서 65세 이상이 차지하는 비율인 고령자 인구 비율이 7% 이상이면 고령화 사회, 14% 이상이면 고령 사회, 20% 이상이면 초고령 사회로 구분된다.

국제 연합 기준(UN)과 국내 국민연금 수급연령(연금 지급개시연령) 및 근로자의 법상 정년(노동관계법령상 정년(60세) 의무규정)을 고려하여 60대를 기준으로 고령화 비율을 나누었다. 

In [11]:
# 격자 id : 격자 폴리곤 매핑 딕셔너리
map_dic = {}
for _, row in filtered_grid.iterrows():
    map_dic[row['properties.gid']] = row['geometry']

In [12]:
geometry_lst = []
for _, row in filtered_respop.iterrows():
    geometry_lst.append(map_dic[row['gid']])
filtered_respop['geometry'] = geometry_lst

https://yganalyst.github.io/spatial_analysis/uber_h3_keplergl/

In [20]:
#  고령화 정도에 따라 grid색상을 지정하는 함수
def rate_color(rate):
    if rate > 20:
        return 'red'
    elif 30 > rate >= 14:
        return 'orange'
    elif 14 > rate >= 7:
        return 'yellow'
    elif 7 > rate > 0:
        return 'green'
    else:
        return 'rgba(255, 0, 0, 0)'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "> &nbsp; 고령화 정도 범례 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 초고령 구역 (60세 이상인구 20% 이상)<br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 고령 구역 (60세 이상인구 20% 이상)<br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 고령화 구역 (60세 이상인구 14% 이상)<br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 일반 구역 (60세 이상인구 7% 이하)<br>
     &nbsp; <i style="background:white">&nbsp;</i>&nbsp; 거주민 없는 구역
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2018 = folium.FeatureGroup(name="2018년", overlay=False) 
m_2019 = folium.FeatureGroup(name="2019년", overlay=False)
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = range(2018, 2023)
layers_lst = [m_2018, m_2019, m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2018부터 2022까지의 연도
    filtered_respop_year = filtered_respop[filtered_respop['연도'] == year]

    # 'gid' 값을 기준으로 그룹화
    for gid in filtered_respop_year['gid'].unique():
        subset = filtered_respop_year[filtered_respop_year['gid'] == gid].copy()
        rate = subset['고령인구_비율'].iloc[0]
        color = rate_color(rate)
        subset['고령인구_비율'] = subset['고령인구_비율'].apply(lambda x: str(x) + '%')

        # subset_popup 데이터프레임을 HTML로 변환하여 popup text에 저장
        subset_popup = subset.iloc[:, 1:-1].style.hide(axis='index').set_properties(
            **{'border': '1px solid black'}).to_html()  # [['연도', '고령인구_비율']]
        popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset_popup}</div>"
        color = rate_color(rate)
        folium.GeoJson(
            filtered_respop_year[filtered_respop_year['gid'] == gid]['geometry'].iloc[0],
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1}
        ).add_to(layers_dic[year]).add_child(folium.Popup(popup_text, max_width=1200), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

### 청주시 성연령별 유동인구

In [15]:
# 원도심 영역(도시재생 대상지역)
floating_population_df = pd.read_csv('SBJ_2309_001/2.청주시_성연령별_유동인구.csv')
floating_population_df['STD_YM'] = floating_population_df['STD_YM'].astype(str)
floating_population_df.rename(columns={'STD_YM':'연도'}, inplace=True)
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
floating_population_df = geo_transform(floating_population_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_floating_population = floating_population_df[floating_population_df['geometry'].within(polygon)]

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [16]:
# 성별 통합한 세대 column 생성
filtered_floating_population['10대'] = filtered_floating_population['m_10g_pop'] + filtered_floating_population['w_10g_pop']
filtered_floating_population['20대'] = filtered_floating_population['m_20g_pop'] + filtered_floating_population['w_20g_pop']
filtered_floating_population['30대'] = filtered_floating_population['m_30g_pop'] + filtered_floating_population['w_30g_pop']
filtered_floating_population['40대'] = filtered_floating_population['m_40g_pop'] + filtered_floating_population['w_40g_pop']
filtered_floating_population['50대'] = filtered_floating_population['m_50g_pop'] + filtered_floating_population['w_50g_pop']
filtered_floating_population['60대'] = filtered_floating_population['m_60g_pop'] + filtered_floating_population['w_60g_pop']

# 세대 통합한 성별 column 생성
filtered_floating_population['남성'] = filtered_floating_population['m_10g_pop'] + filtered_floating_population['m_20g_pop'] + filtered_floating_population['m_30g_pop'] + filtered_floating_population['m_40g_pop'] + filtered_floating_population['m_50g_pop'] + filtered_floating_population['m_60g_pop']
filtered_floating_population['여성'] = filtered_floating_population['w_10g_pop'] + filtered_floating_population['w_20g_pop'] + filtered_floating_population['w_30g_pop'] + filtered_floating_population['w_40g_pop'] + filtered_floating_population['w_50g_pop'] + filtered_floating_population['w_60g_pop']

# 특정 열만 추출하여 성별을 통합한 세대별 유동인구 df생성
filtered_floating_population_gen = filtered_floating_population[list(filtered_floating_population.columns[:1]) + list(filtered_floating_population.columns[16:22]) + [filtered_floating_population.columns[15]]]

# 특정 열만 추출하여  세대를 통합한 성별별 유동인구 df생성
filtered_floating_population_sex = filtered_floating_population[list(filtered_floating_population.columns[:1]) + list(filtered_floating_population.columns[22:24]) + [filtered_floating_population.columns[15]]]

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [17]:
# 최종저장용 df 정의 -> 각 지점들을 grid폴리곤에 매핑시켜 grid별 유동인구로 변환 -> 
filtered_floating_population_totgen = pd.DataFrame()
for grid in tqdm(filtered_grid['geometry'].unique()):
    # 격자와 매핑하기 위해 격자 내에 있는 지점으로만 필터링
    subset = filtered_floating_population_gen[filtered_floating_population_gen['geometry'].within(grid)]
    # 해당 구역 grid df 설정(접합용) 
    grid_df = pd.DataFrame({'geometry' : [grid]})
    # 중간저장용 df 정의
    concat_df = pd.DataFrame()
    
    for date in filtered_floating_population_gen['연도'].unique():
        date_df = pd.DataFrame({'연도' : [date[:4]]}) # 해당 연도 df 설정(접합용) 
        subset_date = subset[subset['연도'] == date]
        # 열 별로 더하기 및 df 변환
        column_sums = subset_date.iloc[:,1:7].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        # 최종 df 접합 및 filtered_floating_population_totgen에 concat
        column_sums_df = pd.concat([date_df, column_sums_df], axis=1)
        concat_df = pd.concat([concat_df, column_sums_df])
    else:
        # 각 연도 별로 평균계산 및 df 변환 -> 월별 유동인구를 평균내었기 때문에, 최종값은 해당 격자의 월별 평균 유동인구가 됨.
        column_means_2020 = concat_df[concat_df['연도'].str.startswith('2020')].iloc[:,1:7].mean()
        date_df_2020 = pd.DataFrame({'연도' : ['2020']})
        column_means_2020_df = pd.DataFrame(column_means_2020).transpose()
        tot_df_2020 = pd.concat([date_df_2020, column_means_2020_df, grid_df], axis=1)
        
        column_means_2021 = concat_df[concat_df['연도'].str.startswith('2021')].iloc[:,1:7].mean()
        date_df_2021 = pd.DataFrame({'연도' : ['2021']})
        column_means_2021_df = pd.DataFrame(column_means_2021).transpose()
        tot_df_2021 = pd.concat([date_df_2021, column_means_2021_df, grid_df], axis=1)
        
        column_means_2022 = concat_df[concat_df['연도'].str.startswith('2022')].iloc[:,1:7].mean()
        date_df_2022 = pd.DataFrame({'연도' : ['2022']})
        column_means_2022_df = pd.DataFrame(column_means_2022).transpose()
        tot_df_2022 = pd.concat([date_df_2022, column_means_2022_df, grid_df], axis=1)
        
        tot_df = pd.concat([tot_df_2020, tot_df_2021, tot_df_2022])
        filtered_floating_population_totgen = pd.concat([filtered_floating_population_totgen, tot_df])

filtered_floating_population_totgen

  0%|          | 0/97 [00:00<?, ?it/s]

100%|██████████| 97/97 [00:15<00:00,  6.42it/s]


,연도,10대,20대,30대,40대,50대,60대,geometry
0,2020,50.681667,127.846667,116.810000,152.100833,141.485000,127.499167,POLYGON ((127.48545865561707 36.63438645254012...
0,2021,76.315000,161.067500,125.623333,161.721667,149.918333,151.808333,POLYGON ((127.48545865561707 36.63438645254012...
0,2022,87.549167,179.480833,138.754167,180.874167,176.260833,202.340833,POLYGON ((127.48545865561707 36.63438645254012...
0,2020,39.828333,117.443333,107.360833,144.343333,156.565000,188.415000,POLYGON ((127.48881487158114 36.63078081970196...
0,2021,45.628333,118.509167,109.384167,159.325000,178.774167,244.094167,POLYGON ((127.48881487158114 36.63078081970196...
...,...,...,...,...,...,...,...,...
0,2021,8.158333,19.206667,15.516667,20.639167,21.222500,22.195833,POLYGON ((127.48545797797155 36.63799244545596...
0,2022,10.209167,20.325833,13.312500,17.649167,18.635833,23.408333,POLYGON ((127.48545797797155 36.63799244545596...
0,2020,7.262500,21.264167,21.885000,29.961667,31.446667,28.694167,POLYGON ((127.49105137605241 36.63528850233756...
0,2021,8.230833,20.689167,21.135000,29.917500,30.412500,27.148333,POLYGON ((127.49105137605241 36.63528850233756...


### 볌례에는 세대별 색상이 있고, 격자를 누르면 plotly나 matplotlib으로 시각화한 그래프가 있어야 할듯

In [19]:
filtered_floating_population_totgen

,연도,10대,20대,30대,40대,50대,60대,geometry
0,2020,50.681667,127.846667,116.810000,152.100833,141.485000,127.499167,POLYGON ((127.48545865561707 36.63438645254012...
0,2021,76.315000,161.067500,125.623333,161.721667,149.918333,151.808333,POLYGON ((127.48545865561707 36.63438645254012...
0,2022,87.549167,179.480833,138.754167,180.874167,176.260833,202.340833,POLYGON ((127.48545865561707 36.63438645254012...
0,2020,39.828333,117.443333,107.360833,144.343333,156.565000,188.415000,POLYGON ((127.48881487158114 36.63078081970196...
0,2021,45.628333,118.509167,109.384167,159.325000,178.774167,244.094167,POLYGON ((127.48881487158114 36.63078081970196...
...,...,...,...,...,...,...,...,...
0,2021,8.158333,19.206667,15.516667,20.639167,21.222500,22.195833,POLYGON ((127.48545797797155 36.63799244545596...
0,2022,10.209167,20.325833,13.312500,17.649167,18.635833,23.408333,POLYGON ((127.48545797797155 36.63799244545596...
0,2020,7.262500,21.264167,21.885000,29.961667,31.446667,28.694167,POLYGON ((127.49105137605241 36.63528850233756...
0,2021,8.230833,20.689167,21.135000,29.917500,30.412500,27.148333,POLYGON ((127.49105137605241 36.63528850233756...


In [24]:
filtered_floating_population_totgen_year

,연도,10대,20대,30대,40대,50대,60대,geometry
0,2022,87.549167,179.480833,138.754167,180.874167,176.260833,202.340833,POLYGON ((127.48545865561707 36.63438645254012...
0,2022,64.359167,143.525000,116.851667,181.030000,208.993333,371.910000,POLYGON ((127.48881487158114 36.63078081970196...
0,2022,43.165000,76.216667,71.404167,111.016667,126.477500,224.040000,POLYGON ((127.48993221161062 36.63979590326395...
0,2022,248.488333,438.350000,348.358333,479.657500,528.889167,696.091667,POLYGON ((127.48881304706971 36.64340179315867...
0,2022,81.849167,166.260000,152.975000,229.585833,253.134167,389.551667,POLYGON ((127.48993244621151 36.63799290759625...
...,...,...,...,...,...,...,...,...
0,2022,7.500833,9.591667,12.325833,20.245833,23.066667,29.376667,POLYGON ((127.48545713074516 36.64249993352423...
0,2022,93.759167,159.185833,118.909167,172.945000,193.390000,258.985000,POLYGON ((127.48881317742963 36.64250029594341...
0,2022,13.775833,33.015833,31.313333,40.080833,38.855833,42.523333,POLYGON ((127.48545780854133 36.63889394334309...
0,2022,10.209167,20.325833,13.312500,17.649167,18.635833,23.408333,POLYGON ((127.48545797797155 36.63799244545596...


In [28]:
subset

,연도,10대,20대,30대,40대,50대,60대,geometry
0,2022,10.925833,25.561667,27.305,40.8625,40.893333,37.958333,POLYGON ((127.49105137605241 36.63528850233756...


In [33]:
subset.iloc[0,1:7]

10대    10.925833
20대    25.561667
30대       27.305
40대      40.8625
50대    40.893333
60대    37.958333
Name: 0, dtype: object

In [39]:
list(subset.columns[1:7])

['10대', '20대', '30대', '40대', '50대', '60대']

In [38]:
subset.iloc[0, 1:7]

10대    10.925833
20대    25.561667
30대       27.305
40대      40.8625
50대    40.893333
60대    37.958333
Name: 0, dtype: object

In [56]:
subset

,연도,10대,20대,30대,40대,50대,60대,geometry
0,2022,10.925833,25.561667,27.305,40.8625,40.893333,37.958333,POLYGON ((127.49105137605241 36.63528850233756...


In [60]:
# 데이터 프레임 재구성
subset_1 = subset.iloc[:, 1:7]
subset_1 = subset_1.transpose()
subset_1 = subset_1.rename(columns={0: '비율'})

# 평균값 계산
average_values = [103.0647680412371, 191.83453608247427, 139.68928694158075, 191.6930068728522, 200.41648625429556, 261.3973883161512]

# 그래프 생성
fig = px.bar(subset_1, x=subset_1.index, y='비율', title='연령별 비율')
fig.update_layout(xaxis_title='연령대', yaxis_title='비율')

# 평균값을 그래프에 추가
#fig.add_trace(px.bar(x=subset_1.index, y=average_values))

fig.show()

In [26]:
column_means_year = filtered_floating_population_totgen_year.iloc[:,1:7].mean()
#date_df_2021 = pd.DataFrame({'연도' : ['2021']})
column_means_year_df = pd.DataFrame(column_means_year).transpose()
column_means_year_df

,10대,20대,30대,40대,50대,60대
0,103.064768,191.834536,139.689287,191.693007,200.416486,261.397388


In [47]:
column_means_year_df.iloc[0,:7].tolist()

[103.0647680412371,
 191.83453608247427,
 139.68928694158075,
 191.6930068728522,
 200.41648625429556,
 261.3973883161512]

In [55]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 grid(격자) 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = filtered_grid['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="격자",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = ['2020', '2021', '2022']
layers_lst = [m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2020부터 2022까지의 연도
    filtered_floating_population_totgen_year = filtered_floating_population_totgen[filtered_floating_population_totgen['연도'] == year]
    # 'gid' 값을 기준으로 그룹화
    for polygon in filtered_floating_population_totgen_year['geometry'].unique():
        subset = filtered_floating_population_totgen_year[filtered_floating_population_totgen_year['geometry'] == polygon].copy()

        # subset_popup 데이터프레임을 HTML로 변환하여 popup text에 저장
        subset_popup = subset.iloc[:, :-1].style.hide(axis='index').set_properties(
            **{'border': '1px solid black'}).to_html()  # [['연도', '고령인구_비율']]
        popup_text = f"<div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset_popup}</div>"
        color = rate_color(rate)
        folium.GeoJson(
            subset['geometry'].iloc[0],
            style_function=lambda feature, color='black': {
                'fillColor': 'black',
                'color': 'black',
                'weight': 1}
        ).add_to(layers_dic[year]).add_child(folium.Popup(popup_text, max_width=1200), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
#m.get_root().html.add_child(folium.Element(legend_html))
m

In [138]:
# 최종저장용 df 정의 -> 각 지점들을 grid폴리곤에 매핑시켜 grid별 유동인구로 변환 -> 
filtered_floating_population_totsex = pd.DataFrame()
for grid in tqdm(filtered_grid['geometry'].unique()):
    # 격자와 매핑하기 위해 격자 내에 있는 지점으로만 필터링
    subset = filtered_floating_population_sex[filtered_floating_population_sex['geometry'].within(grid)]
    # 해당 구역 grid df 설정(접합용) 
    grid_df = pd.DataFrame({'geometry' : [grid]})
    # 중간저장용 df 정의
    concat_df = pd.DataFrame()
    
    for date in filtered_floating_population_sex['연도'].unique():
        date_df = pd.DataFrame({'연도' : [date[:4]]}) # 해당 연도 df 설정(접합용) 
        subset_date = subset[subset['연도'] == date]
        # 열 별로 더하기 및 df 변환
        column_sums = subset_date.iloc[:,1:3].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        # 최종 df 접합 및 filtered_floating_population_totsex에 concat
        column_sums_df = pd.concat([date_df, column_sums_df], axis=1)
        concat_df = pd.concat([concat_df, column_sums_df])
    else:
        # 각 연도 별로 평균계산 및 df 변환
        column_means_2020 = concat_df[concat_df['연도'].str.startswith('2020')].iloc[:,1:3].mean()
        date_df_2020 = pd.DataFrame({'연도' : ['2020']})
        column_means_2020_df = pd.DataFrame(column_means_2020).transpose()
        tot_df_2020 = pd.concat([date_df_2020, column_means_2020_df, grid_df], axis=1)
        
        column_means_2021 = concat_df[concat_df['연도'].str.startswith('2021')].iloc[:,1:3].mean()
        date_df_2021 = pd.DataFrame({'연도' : ['2021']})
        column_means_2021_df = pd.DataFrame(column_means_2021).transpose()
        tot_df_2021 = pd.concat([date_df_2021, column_means_2021_df, grid_df], axis=1)
        
        column_means_2022 = concat_df[concat_df['연도'].str.startswith('2022')].iloc[:,1:3].mean()
        date_df_2022 = pd.DataFrame({'연도' : ['2022']})
        column_means_2022_df = pd.DataFrame(column_means_2022).transpose()
        tot_df_2022 = pd.concat([date_df_2022, column_means_2022_df, grid_df], axis=1)
        
        tot_df = pd.concat([tot_df_2020, tot_df_2021, tot_df_2022])
        filtered_floating_population_totsex = pd.concat([filtered_floating_population_totsex, tot_df])

filtered_floating_population_totsex

100%|██████████| 97/97 [00:14<00:00,  6.65it/s]


,연도,남성,여성,geometry
0,2020,409.335833,307.087500,POLYGON ((127.48545865561707 36.63438645254012...
0,2021,449.799167,376.655000,POLYGON ((127.48545865561707 36.63438645254012...
0,2022,523.735000,441.525000,POLYGON ((127.48545865561707 36.63438645254012...
0,2020,426.297500,327.658333,POLYGON ((127.48881487158114 36.63078081970196...
0,2021,476.890833,378.824167,POLYGON ((127.48881487158114 36.63078081970196...
...,...,...,...,...
0,2021,61.543333,45.395833,POLYGON ((127.48545797797155 36.63799244545596...
0,2022,57.252500,46.288333,POLYGON ((127.48545797797155 36.63799244545596...
0,2020,81.215000,59.299167,POLYGON ((127.49105137605241 36.63528850233756...
0,2021,77.403333,60.130000,POLYGON ((127.49105137605241 36.63528850233756...


### 청주시 폐공장현황

In [ ]:
factory_df = pd.read_csv('SBJ_2309_001/9.청주시_폐공장현황.csv')
factory_df

,addr,reg_dt,site_area,mnfac_area,addfac_area,cncl_dt,cncl_resn,lon,lat
0,충청북도 청주시 서원구 현도면 시목외천로 381,20200527,1029.00,409.830,83.71,20210702,이전에의한폐업,127.414516,36.522164
1,"충청북도 청주시 청원구 율천남로 92, 1층 (내덕동)",20200226,273.00,132.090,0.00,20230222,이전에의한폐업,127.481527,36.662701
2,충청북도 청주시 서원구 현도면 죽암도원로 12-28,20191025,305.22,155.200,150.02,20210913,이전에의한폐업,127.432720,36.520385
3,충청북도 청주시 청원구 북이면 충청대로 1257-3,20181024,0.00,120.000,0.00,20220624,자진폐업,127.548659,36.752720
4,충청북도 청주시 청원구 북이면 용계내추로 22,20140117,1634.00,683.400,97.50,20230313,이전에의한폐업,127.518171,36.749901
...,...,...,...,...,...,...,...,...,...
204,"충청북도 청주시 서원구 남이면 저산척북로 652, 주 2동",20170306,0.00,187.310,0.00,20210706,이전에의한폐업,127.423037,36.573530
205,충청북도 청주시 청원구 오창읍 각리1길 49,20180130,4286.10,2374.785,395.74,20211126,자진폐업,127.433611,36.707257
206,충청북도 청주시 흥덕구 오송읍 쌍청리 52-11번지,20130204,0.00,196.020,0.00,20210127,자진폐업,127.351052,36.640529
207,충청북도 청주시 흥덕구 옥산면 호죽화산로 140-8,20060608,990.00,199.920,0.00,20220517,이전에의한폐업,127.377697,36.704743


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
factory_df = geo_transform(factory_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_factory = factory_df[factory_df['geometry'].within(polygon)]

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_factory.iterrows():
    marker = folium.CircleMarker(
        location=[row['geometry'].y, row['geometry'].x],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 문화재 현황 (폴리곤 영역확장 변경으로 교통분석 후 문화재 필터링 해야할 듯)

https://m.blog.naver.com/zukyun59/222997801975 - 역사문화전경 보존지역 관련 글

In [ ]:
culture_assets_df = pd.read_csv('SBJ_2309_001/5.청주시_유적지_문화재현황.csv')

In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
culture_assets_df = geo_transform(culture_assets_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# 폴리곤을 확장 (예: 0.1 도 또는 10 킬로미터 확장)
expanded_polygon = polygon.buffer(0.015)  # 0.015은 확장할 크기. EPSG 4326 좌표계에서는 위도 및 경도가 0.015도씩 확장됨.(약 1.7km 나중에 변경 될 수 있음)

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_culture = culture_assets_df[culture_assets_df['geometry'].within(expanded_polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_culture

,gbn,clt_nm,addr,lon,lat,geometry
0,국보,청주 용두사지 철당간,충청북도 청주시 상당구 남문로2가 48-19,127.489758,36.633095,POINT (127.48976 36.63309)
1,보물,청주 용화사 석조불상군,충청북도 청주시 서원구 사직동 216-1,127.482003,36.641779,POINT (127.48200 36.64178)
2,충북도 유형문화재,청주 충청도병마절도사영문,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
3,충북도 유형문화재,청주 탑동 오층석탑,충청북도 청주시 상당구 탑동 251,127.497987,36.629600,POINT (127.49799 36.62960)
4,충북도 유형문화재,청주향교,충청북도 청주시 상당구 대성동 68,127.498060,36.637203,POINT (127.49806 36.63720)
5,충북도 유형문화재,청주 청녕각,충청북도 청주시 상당구 북문로1가 171-3,127.488321,36.634704,POINT (127.48832 36.63470)
6,충북도 유형문화재,청주 망선루,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
7,충북도 유형문화재,청주 탑동 양관,충청북도 청주시 상당구 탑동 185-1,127.496569,36.627226,POINT (127.49657 36.62723)
8,충북도 유형문화재,청주 조헌 전장기적비,충청북도 청주시 상당구 남문로2가 92-6,127.488154,36.632669,POINT (127.48815 36.63267)
9,충북도 유형문화재,청주 성공회 성당,충청북도 청주시 상당구 수동 202-1,127.493948,36.638636,POINT (127.49395 36.63864)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_culture.iterrows():
    popup_text = f"{row['gbn']} - {row['clt_nm']}건"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m).add_child(folium.Popup(popup_text, max_width=100))

# 지도 저장
m

### 청주시 시장 현황 (폴리곤 영역확장 없이 범위 내 시장들만 필터링)

In [ ]:
market_df = pd.read_csv('SBJ_2309_001/6.청주시_시장현황.csv')
market_df

,market_nm,addr,area,lon,lat
0,육거리종합시장,상당구 청남로2197번길 42(석교동),86273,127.488184,36.627938
1,서문시장,상당구 남사로89번길 57(서문동),10285,127.485365,36.634894
2,중앙시장,상당구 중앙로 26(북문로2가),7305,127.489192,36.639385
3,사직시장,서원구 사직대로265번길 7(사직동),9200,127.476868,36.636143
4,북부시장,청원구 향군로31번길 19(우암동),22133,127.486221,36.647973


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
market_df = geo_transform(market_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_market = market_df[market_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_market

,market_nm,addr,area,lon,lat,geometry
0,육거리종합시장,상당구 청남로2197번길 42(석교동),86273,127.488184,36.627938,POINT (127.48818 36.62794)
1,서문시장,상당구 남사로89번길 57(서문동),10285,127.485365,36.634894,POINT (127.48536 36.63489)
2,중앙시장,상당구 중앙로 26(북문로2가),7305,127.489192,36.639385,POINT (127.48919 36.63938)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_market.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 주차장 현황 (폴리곤 영역확장 고민해봐야 할듯)

In [ ]:
parking_df = pd.read_csv('SBJ_2309_001/7.청주시_주차장현황.csv')
parking_df

,carpark_nm,carpark_type,addr,slots,lon,lat
0,청주랜드 동물원,노상,충청북도 청주시 상당구 명암로 171,255,127.515747,36.650530
1,상공회의소,노상,충청북도 청주시 상당구 상당로 102,25,127.490487,36.637486
2,명문식당,노상,충청북도 청주시 상당구 우암산로4,30,127.490514,36.639075
3,고려삼계탕,노상,충청북도 청주시 상당구 교동로3번길 48-1,29,127.490837,36.639529
4,메리츠화재,노상,충청북도 청주시 상당구 교동로3번길 127,20,127.490239,36.643059
...,...,...,...,...,...,...
400,견인차보관소,노외,충청북도 청주시 청원구 각리 641-15,193,127.434307,36.705982
401,공한지주차장,노외,양청리 683-3,18,127.434645,36.722995
402,공한지주차장,노외,내덕동 392-14,4,127.488246,36.661356
403,공한지주차장,노외,정하동 35-3,8,127.464401,36.672168


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
parking_df = geo_transform(parking_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_parking = parking_df[parking_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_parking # 노상 : 도로 위 한켠에 주차를 하고 일정 비용을 받음(정부 또는 지자체 관할), 노외 : 공용주차장처럼 별도 대지에 설치되어 제공되는 주차장

,carpark_nm,carpark_type,addr,slots,lon,lat,geometry
0,상공회의소,노상,충청북도 청주시 상당구 상당로 102,25,127.490487,36.637486,POINT (127.49049 36.63749)
1,명문식당,노상,충청북도 청주시 상당구 우암산로4,30,127.490514,36.639075,POINT (127.49051 36.63908)
2,고려삼계탕,노상,충청북도 청주시 상당구 교동로3번길 48-1,29,127.490837,36.639529,POINT (127.49084 36.63953)
3,메리츠화재,노상,충청북도 청주시 상당구 교동로3번길 127,20,127.490239,36.643059,POINT (127.49024 36.64306)
4,교서로(지하상가~주성초교),노상,충청북도 청주시 상당구 교서로 45,49,127.485254,36.641427,POINT (127.48525 36.64143)
5,제일학원 앞,노상,충청북도 청주시 상당구 교동로3번길 153,26,127.490198,36.644232,POINT (127.49020 36.64423)
6,시청후문,노상,충청북도 청주시 상당구 상당로143번길 16,26,127.487858,36.641205,POINT (127.48786 36.64121)
7,청주병원,노상,충청북도 청주시 상당구 중앙로 83,35,127.487734,36.644668,POINT (127.48773 36.64467)
8,한전아파트앞,노상,충청북도 청주시 상당구 사직대로361번길 113,28,127.486806,36.641929,POINT (127.48681 36.64193)
9,청중후문앞,노상,충청북도 청주시 상당구 사직대로361번길 125,46,127.485956,36.642952,POINT (127.48596 36.64295)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_parking.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 공원 현황 (폴리곤 영역확장 고민해봐야 할듯)

In [ ]:
park_df = pd.read_csv('SBJ_2309_001/8.청주시_공원현황.csv')
park_df

,park_gbn,park_nm,lon,lat
0,근린공원,금천배수지,127.513111,36.626271
1,어린이공원,사랑,127.509585,36.629404
2,어린이공원,늘푸름,127.511072,36.625867
3,어린이공원,으뜸,127.510518,36.623173
4,소공원,금남,127.504926,36.622042
5,소공원,쇠내,127.503190,36.625444
6,소공원,꽃산,127.506817,36.625129
7,소공원,쇠내울,127.502813,36.629927
8,소공원,쌈지,127.508663,36.624836
9,체육공원,호미골체육공원,127.512775,36.629755


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
park_df = geo_transform(park_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_park = park_df[park_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_park

,park_gbn,park_nm,lon,lat,geometry
0,소공원,(구)여성회관 앞,127.489598,36.640050,POINT (127.48960 36.64005)
1,어린이공원,남주,127.483619,36.631123,POINT (127.48362 36.63112)
2,테마공원,청주중앙공원,127.487601,36.632739,POINT (127.48760 36.63274)
3,근린공원,상당공원,127.491402,36.636940,POINT (127.49140 36.63694)
4,광장,청주청소년광장,127.488435,36.639388,POINT (127.48843 36.63939)
5,근린공원,옛청주역사공원,127.488923,36.640869,POINT (127.48892 36.64087)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_park.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 학교 현황

In [ ]:
school_df = pd.read_csv('SBJ_2309_001/24.청주시_학교현황.csv')
school_df

,school_nm,estab_gbn,school_chtr,addr,lon,lat
0,금천고등학교,공립,일반고등학교,충청북도 청주시 상당구 호미로233번길 65 금천고등학교 (금천동),127.511943,36.627797
1,대성여자상업고등학교,사립,특성화고등학교,충청북도 청주시 상당구 대성로 220(수동),127.492313,36.645357
2,충북에너지고등학교,공립,특수목적고등학교,충청북도 청주시 상당구 미원면 미원초정로 27(미원면),127.657148,36.639802
3,상당고등학교,공립,일반고등학교,충청북도 청주시 상당구 월평로238번길 3-10(용암동),127.508949,36.606181
4,세광고등학교,사립,일반고등학교,충청북도 청주시 서원구 2순환로 1572(미평동 세광고등학교),127.453040,36.603148
...,...,...,...,...,...,...
171,청주소로초등학교,공립,NaN,충청북도 청주시 흥덕구 옥산면 오송가락로 1020 청주소로초등학교,127.390317,36.672427
172,청주내곡초등학교,공립,NaN,충청북도 청주시 흥덕구 송화로214번길 30 (송절동)청주내곡초등학교,127.457277,36.667208
173,단재초등학교,공립,NaN,충청북도 청주시 상당구 단재로 350 단재초등학교 (방서동),127.500160,36.599530
174,생명초등학교,공립,NaN,충청북도 청주시 청원구 오창읍 중부로 772(오창읍),127.438795,36.711987


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
school_df = geo_transform(school_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_school = school_df[school_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_school

,school_nm,estab_gbn,school_chtr,addr,lon,lat,geometry
0,청주공업고등학교,공립,특성화고등학교,충청북도 청주시 상당구 교서로 17(영동),127.485500,36.638590,POINT (127.48550 36.63859)
1,청주중학교,공립,NaN,충청북도 청주시 상당구 사직대로361번길 125(영동청주중학교),127.485875,36.643274,POINT (127.48587 36.64327)
2,주성초등학교,공립,NaN,충청북도 청주시 상당구 교서로 45 주성초등학교 (영동 주성초등학교),127.485200,36.641424,POINT (127.48520 36.64142)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_school.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 어린이집 유치원 현황

In [ ]:
kinder_df = pd.read_csv('SBJ_2309_001/28.청주시_어린이집_유치원현황.csv')
kinder_df

,fac_nm,fxd_num,prsnt_num,lon,lat
0,행정초등학교병설유치원,17,5,127.531947,36.536620
1,남일초등학교병설유치원,17,10,127.509473,36.575787
2,청남어린이집,89,41,127.512457,36.576130
3,성무어린이집,88,55,127.513181,36.580777
4,경원어린이집,96,72,127.509341,36.586107
...,...,...,...,...,...
674,푸른하늘어린이집,20,18,127.418087,36.711524
675,비봉유치원,139,109,127.413230,36.712757
676,자연어린이집,231,78,127.412753,36.718686
677,세계어린이집,96,66,127.408251,36.727684


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
kinder_df = geo_transform(kinder_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_kinder = kinder_df[kinder_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_kinder

,fac_nm,fxd_num,prsnt_num,lon,lat,geometry
0,센트럴칸타빌어린이집,42,38,127.492188,36.633124,POINT (127.49219 36.63312)
1,주성초등학교병설유치원,17,13,127.485254,36.641427,POINT (127.48525 36.64143)
2,성안나유치원,216,76,127.491409,36.641805,POINT (127.49141 36.64181)
3,은물영아전담어린이집,32,23,127.490870,36.642238,POINT (127.49087 36.64224)
4,코아루휴티스어린이집,34,30,127.489458,36.643841,POINT (127.48946 36.64384)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_kinder.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 공공기관 및 주요지점 현황

In [ ]:
public_df = pd.read_csv('SBJ_2309_001/25.청주시_공공기관_및_주요지점현황.csv')
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
public_df = geo_transform(public_df)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_public = public_df[public_df['geometry'].within(polygon)].reset_index(drop=True)

In [ ]:
public_df

,inst_gbn,inst_nm,lon,lat,geometry
0,시군청,청주시 상당보건소,127.492925,36.641236,POINT (127.49293 36.64124)
1,시군청,청주 흥덕경찰서,127.454965,36.656568,POINT (127.45496 36.65657)
2,시군청,청주 동부소방서,127.494864,36.620987,POINT (127.49486 36.62099)
3,시군청,청주 서부소방서,127.437042,36.627264,POINT (127.43704 36.62726)
4,시군청,청주세무서,127.439759,36.638886,POINT (127.43976 36.63889)
...,...,...,...,...,...
164,민간기관,쥬네쓰쇼핑몰,127.488134,36.635258,POINT (127.48813 36.63526)
165,도단위기관,충청북도청소년종합진흥원,127.492273,36.634533,POINT (127.49227 36.63453)
166,금융기관,수협은행 청주금융센터,127.491114,36.632575,POINT (127.49111 36.63258)
167,금융기관,SC제일은행,127.488464,36.635645,POINT (127.48846 36.63564)


In [ ]:
filtered_points_public

,inst_gbn,inst_nm,lon,lat,geometry
0,시군청,청주우체국,127.488969,36.633807,POINT (127.48897 36.63381)
1,시군청,중앙동주민센터,127.488198,36.639856,POINT (127.48820 36.63986)
2,시군청,청주시의회,127.489673,36.641613,POINT (127.48967 36.64161)
3,금융기관,우리은행 청주지점,127.487831,36.631849,POINT (127.48783 36.63185)
4,금융기관,신한은행 청주지점,127.489080,36.631181,POINT (127.48908 36.63118)
5,금융기관,수협중앙회 청주지점,127.490309,36.639388,POINT (127.49031 36.63939)
6,금융기관,농협 충북지역본부,127.489332,36.643592,POINT (127.48933 36.64359)
7,금융기관,외환은행 청주지점,127.491314,36.631163,POINT (127.49131 36.63116)
8,금융기관,한국산업은행 청주지점,127.488861,36.635399,POINT (127.48886 36.63540)
9,공공기관,한국감정원 청주지점,127.489525,36.634582,POINT (127.48952 36.63458)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_public.iterrows():
    popup_text = f"{row['inst_gbn']} - {row['inst_nm']}"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m).add_child(folium.Popup(popup_text, max_width=200))

# 지도 저장
m

### 청주시 가로수 현황 (공공데이터포털 - 산림청_도시숲가로수관리 가로수 현황 data https://www.data.go.kr/data/15120900/fileData.do)

In [ ]:
'''산림청_도시숲가로수관리 가로수 현황_20221231 데이터 정제 완료 -'''
# street_tree_df = pd.read_csv('SBJ_2309_001/산림청_도시숲가로수관리 가로수 현황_20221231.csv', encoding='cp949')
# street_tree_df = street_tree_df[['시군구명', '도로구간명', '수종명', '지역X좌표', '지역Y좌표']]
# street_tree_df = street_tree_df[street_tree_df['시군구명'].str.contains('청주')].reset_index(drop=True)

# # EPSG 5186에서 EPSG 4326으로 좌표 변환을 수행하는 함수 생성
# def transform_coordinates(row):
#     in_proj = pyproj.Proj(init='epsg:5186')
#     out_proj = pyproj.Proj(init='epsg:4326')
#     lon, lat = pyproj.transform(in_proj, out_proj, row['지역X좌표'], row['지역Y좌표'])
#     return pd.Series({'lat': lat, 'lon': lon})

# # '지역X좌표'와 '지역Y좌표'를 EPSG 4326으로 변환하고 'lat' 및 'lon' 열로 이름 변경
# street_tree_df[['lat', 'lon']] = street_tree_df.apply(transform_coordinates, axis=1)
# # '지역X좌표'와 '지역Y좌표' 열을 삭제 (선택 사항)
# street_tree_df.drop(['지역X좌표', '지역Y좌표'], axis=1, inplace=True)

# 

'산림청_도시숲가로수관리 가로수 현황_20221231 데이터 정제 완료 -'

In [ ]:
street_tree_df = pd.read_csv('SBJ_2309_001/29.청주시_가로수현황.csv', encoding='UTF8')
street_tree_df

,시군구명,도로구간명,수종명,lat,lon
0,충청북도 청주시 청원구,오창대로,벚나무,36.708975,127.481089
1,충청북도 청주시 청원구,오창대로,벚나무,36.708883,127.480707
2,충청북도 청주시 청원구,오창대로,벚나무,36.708996,127.481787
3,충청북도 청주시 청원구,오창대로,벚나무,36.708625,127.480314
4,충청북도 청주시 청원구,오창대로,벚나무,36.708993,127.481158
...,...,...,...,...,...
24481,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660599,127.507634
24482,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660668,127.507612
24483,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660737,127.507589
24484,충청북도 청주시 청원구,율중로29번길,이팝나무,36.660249,127.507748


In [ ]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
street_tree_df = geo_transform(street_tree_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_tree = street_tree_df[street_tree_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
filtered_points_tree

,시군구명,도로구간명,수종명,lat,lon,geometry
0,충청북도 청주시 상당구,교서로,은행나무,36.637265,127.486389,POINT (127.48639 36.63726)
1,충청북도 청주시 상당구,남사로,은행나무,36.632017,127.487616,POINT (127.48762 36.63202)
2,충청북도 청주시 상당구,남사로,은행나무,36.632060,127.489971,POINT (127.48997 36.63206)
3,충청북도 청주시 상당구,남사로,은행나무,36.632081,127.488882,POINT (127.48888 36.63208)
4,충청북도 청주시 상당구,남사로,은행나무,36.632098,127.489197,POINT (127.48920 36.63210)
...,...,...,...,...,...,...
454,충청북도 청주시 상당구,상당로70번길,칠엽수,36.634603,127.491657,POINT (127.49166 36.63460)
455,충청북도 청주시 상당구,상당로70번길,칠엽수,36.634618,127.491736,POINT (127.49174 36.63462)
456,충청북도 청주시 상당구,상당로81번길,버즘나무,36.635674,127.490370,POINT (127.49037 36.63567)
457,충청북도 청주시 상당구,상당로81번길,버즘나무,36.635540,127.490390,POINT (127.49039 36.63554)


In [ ]:
from folium.plugins import MarkerCluster
# Folium 지도 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=13, width='100%', height='100%')

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# GeoJSON 데이터를 지도에 추가
for index, row in filtered_points_tree.iterrows():
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    )
    marker.add_to(m)

# 지도 저장
m

### 청주시 상세 도로망 시각화

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/13.청주시_상세도로망.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : LineString(x))
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.02) # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadsystem = roadsystem_df[roadsystem_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_roadsystem

,type,properties.link_id,properties.max_speed,properties.road_name,properties.road_no,properties.road_rank,properties.link_type,properties.pavement,properties.road_type,properties.facil_name,properties.up_lanes,properties.dw_lanes,properties.oneway,properties.length,properties.up_its_id,properties.dw_its_id,geometry.type,geometry.coordinates,geometry
0,Feature,564400018,30,구룡산로371번길,0,107,32768,0,0,None,1,1,0,0.133,2830113400,2830113500,LineString,"[[127.48562418391985, 36.618276576111015], [12...","LINESTRING (127.48562 36.61828, 127.48570 36.6..."
1,Feature,564400030,50,교동로3번길,0,107,32768,0,0,None,1,0,1,0.218,2700148100,0,LineString,"[[127.49072640629673, 36.6372799342449], [127....","LINESTRING (127.49073 36.63728, 127.49064 36.6..."
2,Feature,564400040,50,중앙로,0,107,4,0,0,None,1,1,0,0.050,2840061500,2840061400,LineString,"[[127.48794851415629, 36.647618175921004], [12...","LINESTRING (127.48795 36.64762, 127.48801 36.6..."
3,Feature,564400062,0,향군로,0,107,4,0,0,None,1,0,1,0.029,0,0,LineString,"[[127.48801474794416, 36.64716757354723], [127...","LINESTRING (127.48801 36.64717, 127.48829 36.6..."
4,Feature,564400099,30,상당로26번길,0,107,32768,0,0,None,1,1,0,0.167,2700160200,2700160100,LineString,"[[127.49296814940611, 36.63064782192794], [127...","LINESTRING (127.49297 36.63065, 127.49286 36.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,Feature,565409729,50,중고개로337번길,0,107,64,0,0,None,2,0,1,0.007,2700049700,0,LineString,"[[127.49903115788317, 36.62493774141136], [127...","LINESTRING (127.49903 36.62494, 127.49903 36.6..."
897,Feature,565409730,0,중고개로337번길,0,107,64,0,0,None,2,0,1,0.025,0,0,LineString,"[[127.49892275143704, 36.624735640427424], [12...","LINESTRING (127.49892 36.62474, 127.49898 36.6..."
898,Feature,565409732,50,산성로,0,107,4,0,0,None,2,0,1,0.022,2700145300,0,LineString,"[[127.49894096542923, 36.62526996720331], [127...","LINESTRING (127.49894 36.62527, 127.49880 36.6..."
899,Feature,565409733,30,산성로9번길,0,107,64,0,0,None,2,0,1,0.014,2700080300,0,LineString,"[[127.4987983771455, 36.6251127834328], [127.4...","LINESTRING (127.49880 36.62511, 127.49876 36.6..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadsystem.iterrows():
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m)
    
m

### 청주시 불법주정차

In [ ]:
illegal_parking_df = pd.read_csv('SBJ_2309_001/14.청주시_불법주정차단속통계.csv')
illegal_parking_df = geo_transform(illegal_parking_df)
illegal_parking_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,cd_type,cd_loc,cd_dong,cd_cnt,lon,lat,geometry
0,고정형,상당로 남궁요양병원사거리부근1,문화동,13,127.490864,36.632164,POINT (127.49086 36.63216)
1,고정형,사직대로 대현지하상가나도약국부근,북문로1가,15,127.487698,36.636797,POINT (127.48770 36.63680)
2,고정형,사직대로 지하상가성안길입구부근,북문로2가,17,127.488849,36.636973,POINT (127.48885 36.63697)
3,고정형,상당로 도청사거리부근,북문로1가,20,127.490597,36.634411,POINT (127.49060 36.63441)
4,고정형,무심동로 육거리주차장부근,석교동,20,127.487675,36.627044,POINT (127.48768 36.62704)
5,고정형,청소년광장사거리부근,북문로2가,272,127.488643,36.639136,POINT (127.48864 36.63914)
6,고정형,홈플러스후문사거리부근,서문동,374,127.488256,36.635447,POINT (127.48826 36.63545)
7,고정형,상당로81번길 산업은행청주점부근,북문로1가,133,127.488747,36.635504,POINT (127.48875 36.63550)
8,고정형,사직대로350번길 청주상호저축은행부근,서문동,129,127.486397,36.634354,POINT (127.48640 36.63435)
9,고정형,교서로 청주공고부근,영동,52,127.486449,36.637435,POINT (127.48645 36.63744)


In [ ]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=12)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

for _, row in illegal_parking_df.iterrows() :
    popup_text = f"{row['cd_loc']}({row['cd_dong']} - {row['cd_cnt']}건)"
    folium.Circle(location=(row['lat'], row['lon']), radius=30, color='#FF580B',
            fill='#FF580B').add_to(m).add_child(folium.Popup(popup_text, max_width=200))
    
m

### 청주시 112신고 통계 (격자매핑 형식)

In [ ]:
police_report_df = pd.read_csv('SBJ_2309_001/20.청주시_112신고통계.csv')
# 5개년 신고건수 총합
police_report_df['repo_total'] = police_report_df['repo_2018'] + police_report_df['repo_2019'] + police_report_df['repo_2020'] + police_report_df['repo_2021'] + police_report_df['repo_2022']
police_report_df['repo_total'] = police_report_df['repo_total'].astype(int)
police_report_df = police_report_df[['gid', 'repo_total']]

#원도심 영역 grid id 리스트 로드 및 police_report_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_police_report = police_report_df[police_report_df['gid'].isin(grid_id)]
filtered_police_report['geometry'] = filtered_grid['geometry'].tolist()
filtered_police_report.sort_values(by='repo_total', ascending=False, inplace=True)
filtered_police_report.reset_index(drop=True, inplace=True)
filtered_police_report

C:\Users\user\AppData\Local\Temp\ipykernel_6272\1370399623.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_police_report['geometry'] = filtered_grid['geometry'].tolist()
C:\Users\user\AppData\Local\Temp\ipykernel_6272\1370399623.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_police_report.sort_values(by='repo_total', ascending=False, inplace=True)


,gid,repo_total,geometry
0,다바989482,1519,POLYGON ((127.48769621544852 36.63168220843648...
1,다바988483,1510,"POLYGON ((127.4865775332584 36.63258358652596,..."
2,다바992477,1406,POLYGON ((127.49105231410148 36.62717501134686...
3,다바991478,953,POLYGON ((127.48993373614363 36.62807642164767...
4,다바990486,887,POLYGON ((127.4888142201003 36.635288313298695...
...,...,...,...
268,다바994489,2,"POLYGON ((127.49328829744827 36.637993143918, ..."
269,다바996484,2,POLYGON ((127.49552579225157 36.63348575733207...
270,다바984494,2,POLYGON ((127.48210108412849 36.64249947656094...
271,다바995485,1,POLYGON ((127.49440717516781 36.63438720867091...


#### 경찰신고수는 평균에 대해 얼마나 높은가를 구해서 시각화 해야할듯...?

KeplerGl로 시각화 해보기

In [ ]:
len(filtered_police_report)

273

In [ ]:
# 데이터프레임에서 데이터 추출
lst = filtered_police_report['repo_total']

# 조건에 따라 각 막대의 색상을 설정
colors = ['green' if x > 600 else 'yellow' if x > 300 else 'red' for x in lst]

# 히스토그램을 생성
fig = go.Figure(go.Histogram(x=lst, marker_color=colors, nbinsx=1000))

# 레이아웃 설정
fig.update_xaxes(title_text='report_count')
fig.update_yaxes(title_text='count')
fig.update_layout(title_text='report_count_histogram')

# 플롯리 그래프 보이기
fig.show()

In [ ]:
# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=12)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 신고건수에 따라 색상을 지정하는 함수
def color_producer(repo_total):
    if repo_total < 300:
        return 'green'
    elif 300 <= repo_total < 800:
        return 'yellow'
    else:
        return 'red'
        
for _, row in filtered_police_report.iterrows() :
    popup_text = f"격자:{row['gid']}(신고건수 - {row['repo_total']}건)"
    color = color_producer(row['repo_total'])
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 0.2}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
m

### 청주시 카드매출 (격자매핑)

KB국민카드 데이터를 활용하기 보다는 총 매출 추정금액을 활용하는 것이 더 효율적일 것으로 예상됨

In [ ]:
cardsales_df = pd.read_csv('SBJ_2309_001/19.청주시_카드매출_격자매핑.csv')
cardsales_df.dropna(inplace=True)
cardsales_df.reset_index(drop=True, inplace=True)
cardsales_df['est_sales'] = (cardsales_df['est_sales'] * 1000).astype(int)
cardsales_df
#원도심 영역 grid id 리스트 로드 및 police_report_df 필터링
grid_id = filtered_grid['properties.gid'].tolist()
filtered_cardsales = cardsales_df[cardsales_df['gid'].isin(grid_id)]
# 'est_sales' 열의 값이 0인 행을 'card_sales' 값으로 채우기
filtered_cardsales['est_sales'] = filtered_cardsales.apply(lambda row: row['card_sales'] if row['est_sales'] == 0 else row['est_sales'], axis=1)
filtered_cardsales

C:\Users\user\AppData\Local\Temp\ipykernel_6272\4040690819.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,stdr_ym,kbc_bzc_nm_1,kbc_bzc_nm_2,kbc_bzc_nm_3,card_sales,est_sales,gid
0,202305,서비스,(정기)납부/대여서비스,미디어/음향,989000,18995000,다바989487
1,202305,서비스,(정기)납부/대여서비스,도서/음반대여,54000,844000,다바989487
2,202305,서비스,의료서비스,한의원,5019200,83333000,다바992490
3,202305,소매업,의류,여성의류,1600000,25979000,다바990484
4,202305,소매업,패션잡화,구두제화,103500,3315000,다바989483
...,...,...,...,...,...,...,...
13006,202306,소매업,의류,일반의류(기타),1657400,1657400,다바990481
13007,202305,음식,커피/음료,커피전문점,98200,98200,다바987489
13008,202306,소매업,패션잡화,액세서리,109010,109010,다바988486
13009,202305,음식,기타외국식,쌀국수전문점,8000,8000,다바991482


In [ ]:
geometry_lst = []
for _, row in filtered_cardsales.iterrows():
    geometry_lst.append(map_dic[row['gid']])
filtered_cardsales['geometry'] = geometry_lst

C:\Users\user\AppData\Local\Temp\ipykernel_6272\1955721771.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
filtered_cardsales['stdr_ym'].unique()

array([202305, 202306, 202301, 202303, 202304, 202302], dtype=int64)

In [ ]:
filtered_cardsales_month = filtered_cardsales[filtered_cardsales['stdr_ym'] == 202302]

In [ ]:
# 'gid' 값을 기준으로 그룹화하고 'est_sales' 합계 리스트 미리 생성, 상위 33%, 중위 33%, 하위 33%의 임계값 탐색
est_lst = []

for gid in filtered_cardsales_month['gid'].unique():
    subset = filtered_cardsales_month[filtered_cardsales_month['gid'] == gid].copy()
    subset['kbc_total'] = subset.iloc[:, 1] + ' - ' +  subset.iloc[:, 3]
    est_sales_sum = subset['est_sales'].sum()
    est_lst.append(est_sales_sum)

# 데이터를 정렬
sorted_sales = sorted(est_lst)

# 상위 33%, 중위 33%, 하위 33%의 임계값을 찾음
num_data_points = len(est_lst)

# 하위 33%와 상위 33%에 해당하는 임계값 계산
lower_threshold = sorted_sales[int(len(sorted_sales) * 0.33)]
upper_threshold = sorted_sales[int(len(sorted_sales) * 0.67)]

#  매출 액수에 따라 색상을 지정하는 함수
def color_picker(est_sales_sum):
    if est_sales_sum < lower_threshold:
        return 'green'
    elif lower_threshold <= est_sales_sum < upper_threshold:
        return 'yellow'
    else:
        return 'red'

In [ ]:
# 범례 생성
legend_html = """
     <div style="position: fixed; 
     top: 50px; right: 50px; width: 500px; height: 125px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.8);
     "><br> &nbsp; 총매출 범례 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 총매출 상위 33% ({1:,}원 이상)<br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 총매출 중위 33% ({0:,}원 이상 & {1:,}원 미만)<br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 총매출 하위 33% ({0:,}원 이하)<br>
     </div>
     """.format(lower_threshold, upper_threshold)

# map 생성
m = folium.Map(location=[36.627797, 127.511943],  zoom_start=13)

# 배경지도 타일 설정하기
layer = "Hybrid"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 'gid' 값을 기준으로 그룹화
for gid in filtered_cardsales_month['gid'].unique():
    subset = filtered_cardsales_month[filtered_cardsales_month['gid'] == gid].copy()
    subset['kbc_total'] = subset.iloc[:, 1] + ' - ' +  subset.iloc[:, 3]
    est_sales_sum = subset['est_sales'].sum()
    
    # 매출 데이터를 3자리마다 쉼표로 구분하고 "원"을 추가
    subset['카테고리'] = subset['kbc_total'].copy()
    subset['총매출'] = subset['est_sales'].apply(lambda x: f'{x:,}원')
        
    # subset_popup 데이터프레임을 HTML로 변환하여 popup text에 저장
    subset_popup = subset[['카테고리', '총매출']].style.hide(axis='index').set_properties(**{'border': '1px solid black'}).to_html()
    popup_text = f"격자 ID - {gid}<br>총매출 - {est_sales_sum:,}원<br><br><div style='max-height: 200px; max-width: 700px; overflow-y: auto;'>{subset_popup}</div>"
    color = color_picker(est_sales_sum)
    folium.GeoJson(
        filtered_cardsales_month[filtered_cardsales_month['gid'] == gid]['geometry'].iloc[0],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 0.5}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=1200))

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))

m

### 청주시 도로명주소(건물 + 도로)

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/15.청주시_도로명주소(건물).geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_building_df = pd.json_normalize(geojson_data['features'])
roadname_building_df = roadname_building_df[['properties.BD_MGT_SN', 'properties.GRO_FLO_CO', 'properties.UND_FLO_CO', 'geometry.coordinates']]
roadname_building_df['properties.GRO_FLO_CO'] = roadname_building_df['properties.GRO_FLO_CO'].astype(int)
roadname_building_df['properties.UND_FLO_CO'] = roadname_building_df['properties.UND_FLO_CO'].astype(int)
roadname_building_df['geometry'] = roadname_building_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [ ]:
roadname_building_df

,properties.BD_MGT_SN,properties.GRO_FLO_CO,properties.UND_FLO_CO,geometry.coordinates,geometry
0,4311331027200100001000001,1,0,"[[[127.36472483620427, 36.591653472471705], [1...",POLYGON ((127.36472483620427 36.59165347247170...
1,4311311500109800001000001,4,0,"[[[127.46487147229752, 36.64026224932595], [12...",POLYGON ((127.46487147229752 36.64026224932595...
2,4311232024104830000000001,1,0,"[[[127.4313024538128, 36.50196125872578], [127...","POLYGON ((127.4313024538128 36.50196125872578,..."
3,4311425327200300000000001,1,0,"[[[127.45834002968785, 36.757888807263626], [1...",POLYGON ((127.45834002968785 36.75788880726362...
4,4311425327200300000000002,1,0,"[[[127.45902902095496, 36.758077329342385], [1...",POLYGON ((127.45902902095496 36.75807732934238...
...,...,...,...,...,...
168838,4311132024200550001000001,1,1,"[[[127.66283486377729, 36.656696110146584], [1...",POLYGON ((127.66283486377729 36.65669611014658...
168839,4311231032200690012000001,2,0,"[[[127.40371399383083, 36.59237314489042], [12...",POLYGON ((127.40371399383083 36.59237314489042...
168840,4311231034102300004000001,2,0,"[[[127.44238535592501, 36.59304242939975], [12...",POLYGON ((127.44238535592501 36.59304242939975...
168841,4311425027200090030000001,2,0,"[[[127.56369191508364, 36.705662793568244], [1...",POLYGON ((127.56369191508364 36.70566279356824...


In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
roadname_building_df = gpd.GeoDataFrame(roadname_building_df, geometry='geometry')

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadname_building = roadname_building_df[roadname_building_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_roadname_building

,properties.BD_MGT_SN,properties.GRO_FLO_CO,properties.UND_FLO_CO,geometry.coordinates,geometry
0,4311110100100600000061755,5,0,"[[[127.48694447846057, 36.6400016341025], [127...","POLYGON ((127.48694 36.64000, 127.48694 36.639..."
1,4311110100100550002032236,1,0,"[[[127.48632680823786, 36.64052917039099], [12...","POLYGON ((127.48633 36.64053, 127.48633 36.640..."
2,4311110100100560003032233,2,0,"[[[127.48655786043167, 36.640535159309565], [1...","POLYGON ((127.48656 36.64054, 127.48646 36.640..."
3,4311110100100560003071386,1,0,"[[[127.48653912832565, 36.640421876670125], [1...","POLYGON ((127.48654 36.64042, 127.48646 36.640..."
4,4311110100100560002032235,3,0,"[[[127.48657086929198, 36.64041777756533], [12...","POLYGON ((127.48657 36.64042, 127.48656 36.640..."
...,...,...,...,...,...
3184,4311110800100300004000001,4,0,"[[[127.49253748307129, 36.63019961400344], [12...","POLYGON ((127.49254 36.63020, 127.49279 36.630..."
3185,4311110100100040003046395,1,0,"[[[127.48581567013639, 36.644730907282785], [1...","POLYGON ((127.48582 36.64473, 127.48588 36.644..."
3186,4311110800100590001049620,5,0,"[[[127.49155463874942, 36.63035531760507], [12...","POLYGON ((127.49155 36.63036, 127.49155 36.630..."
3187,4311110900101650001000001,3,0,"[[[127.48496189891529, 36.63447898239693], [12...","POLYGON ((127.48496 36.63448, 127.48514 36.634..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadname_building.iterrows():
    popup_text = f"지상:{row['properties.GRO_FLO_CO']}층, 지하:{row['properties.UND_FLO_CO']}층)"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

https://thlee33.medium.com/kepler-gl%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EA%B1%B4%EB%AC%BC-3d-%EC%8B%9C%EA%B0%81%ED%99%94-330400887fe3

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/16.청주시_도로명주소(도로).geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_road_df = pd.json_normalize(geojson_data['features'])
roadname_road_df['geometry'] = roadname_road_df['geometry.coordinates'].apply(lambda x: LineString(x))
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadname_road_df = gpd.GeoDataFrame(roadname_road_df, geometry='geometry')

In [ ]:
roadname_road_df

,type,properties.SIG_CD,properties.RDS_MAN_NO,properties.RN,properties.RN_CD,properties.ENG_RN,properties.NTFC_DE,properties.WDR_RD_CD,properties.ROA_CLS_SE,properties.RDS_DPN_SE,properties.RBP_CN,properties.REP_CN,properties.ROAD_BT,properties.ROAD_LT,properties.BSI_INT,properties.ALWNC_RESN,properties.ALWNC_DE,properties.MVM_RES_CD,properties.MVMN_RESN,properties.MVMN_DE,properties.OPERT_DE,geometry.type,geometry.coordinates,geometry
0,Feature,43110,48962,서부로,3014066,Seobu-ro,20100423,1,3,1,현암동 6-2,남이면 상발리 1-2,4.0,497.510,20,청주시 서쪽에서 진입하는 도로,20100423,12,도로구간변경,20100423,20220428152317,LineString,"[[127.40046912223588, 36.59407481769434], [127...","LINESTRING (127.40047 36.59407, 127.40049 36.5..."
1,Feature,43114,37882,오창가좌4길,4520481,Ochanggajwa 4-gil,20090506,3,4,1,오창읍 가좌리 577-0,오창읍 가좌리 산24-0,3.0,230.829,20,행정구역명(오창 가좌)을 이용하여 오창가좌4길로 부여,20090506,12,None,20170426,20170426170627,LineString,"[[127.37870667484435, 36.74003318366039], [127...","LINESTRING (127.37871 36.74003, 127.37869 36.7..."
2,Feature,43114,45102,오창대로,2014001,Ochang-daero,20090717,3,2,1,오창읍 농소리 산15-3,오창읍 농소리 96-22,6.0,172.150,20,청원구 오창 중심도로로 청주국제 공항을 연결하는 도로,20090717,12,도로구간변경,20191126,20191126165702,LineString,"[[127.44062297400885, 36.7066553369069], [127....","LINESTRING (127.44062 36.70666, 127.44062 36.7..."
3,Feature,43113,23851,현암로70번길,4512075,Hyeonam-ro 70beon-gil,20100205,3,4,0,현암동 277,현암동 10-1,3.0,477.000,10,현암로의 시작지점에서부터 약 700m지점에서 오른쪽으로 분기되는 도로,20100205,90,"광역도로 정비 및 부여사유, 부여방식 정비",20141030,20141030224227,LineString,"[[127.38818269925856, 36.590901928084186], [12...","LINESTRING (127.38818 36.59090, 127.38820 36.5..."
4,Feature,43111,23149,단재로95번길,4508164,Danjae-ro 95beon-gil,20100205,3,4,1,영운동 127-2,금천동 154-15,8.0,177.000,10,단재로의 시작지점에서부터 약 950m지점에서 왼쪽으로 분기되는 도로,20100205,90,"광역도로 정비 및 부여사유, 부여방식 정비",20141030,20141030224127,LineString,"[[127.49891142810951, 36.62144490947463], [127...","LINESTRING (127.49891 36.62144, 127.49901 36.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10835,Feature,43113,49483,궁평3길,4520093,Gungpyeong 3-gil,20090506,3,4,1,오송읍 궁평리 545-1,오송읍 궁평리 203-16,6.0,92.363,20,행정구역명(궁평)을 이용하여 궁평3길로 부여,20090506,12,도로구간변경,20090506,20220817091440,LineString,"[[127.3438015310587, 36.629717294522244], [127...","LINESTRING (127.34380 36.62972, 127.34380 36.6..."
10836,Feature,43113,49922,정중2길,4520585,Jeongjung 2-gil,20090506,3,4,1,오송읍 정중리 202-1,오송읍 정중리 산70-28,8.0,310.991,20,행정구역명(정중)을 이용하여 정중2길로 부여,20090506,12,도로구간변경,20090506,20230320103311,LineString,"[[127.3006984747814, 36.622090468383526], [127...","LINESTRING (127.30070 36.62209, 127.30070 36.6..."
10837,Feature,43114,49762,학암옥수길,4520665,Hagamoksu-gil,20090506,3,4,1,북이면 옥수리 597-0,북이면 옥수리 597-0,4.0,280.579,20,행정구역 북이면 추학리 학암마을과 옥수리를 연결하는 도로,20090506,12,도로구간변경,20090506,20230113162509,LineString,"[[127.55442665143991, 36.77324528980817], [127...","LINESTRING (127.55443 36.77325, 127.55442 36.7..."
10838,Feature,43114,34623,충청내륙로,3350598,Chungcheongnaeryuk-ro,20230519,2,3,0,충청북도 청주시 청원구 율량동 167-2 (1),충청북도 청주시 청원구 북이면 금대리 87-1 (1378),20.0,21729.000,20,충청내륙고속화도로 (1-1)공구 준공에 따른 충청내륙로 종점 연장,20140321,12,도로구간변경,20230519,20230524094419,LineString,"[[127.51205901560849, 36.6620270340752], [127....","LINESTRING (127.51206 36.66203, 127.51368 36.6..."


In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadname_road = roadname_road_df[roadname_road_df['geometry'].within(polygon)].reset_index(drop=True)

In [ ]:
filtered_roadname_road

,type,properties.SIG_CD,properties.RDS_MAN_NO,properties.RN,properties.RN_CD,properties.ENG_RN,properties.NTFC_DE,properties.WDR_RD_CD,properties.ROA_CLS_SE,properties.RDS_DPN_SE,properties.RBP_CN,properties.REP_CN,properties.ROAD_BT,properties.ROAD_LT,properties.BSI_INT,properties.ALWNC_RESN,properties.ALWNC_DE,properties.MVM_RES_CD,properties.MVMN_RESN,properties.MVMN_DE,properties.OPERT_DE,geometry.type,geometry.coordinates,geometry
0,Feature,43111,23026,상당로55번길,4508329,Sangdang-ro 55beon-gil,20100205,3,4,1,문화동 104-13,남문로2가 92-6,4.0,40.0,10,상당로의 시작지점에서부터 약 550m지점에서 왼쪽으로 분기되는 도로,20100205,90,"광역도로 정비 및 부여사유, 부여방식 정비",20141030,20141030224133,LineString,"[[127.48852797597128, 36.632931559201296], [12...","LINESTRING (127.48853 36.63293, 127.48853 36.6..."
1,Feature,43111,23043,사직대로,3236025,Sajik-daero,20100205,3,2,1,복대동 966-0,북문로2가 40-1,5.0,3850.0,10,"옛 사직단의 위치해 있음을 인용하고 청주의 상징적 의미의 ""대로""반영",20100205,99,직권수정(속성변경),20141030,20161005094910,LineString,"[[127.48836421819995, 36.63692034002677], [127...","LINESTRING (127.48836 36.63692, 127.48835 36.6..."
2,Feature,43111,23106,사직대로362번길,4508271,Sajik-daero 362beon-gil,20100205,3,4,1,서문동 224-6,서문동 224-6,3.0,50.0,10,"사직대로의 시작지점에서부터 약 3,620m지점에서 오른쪽으로 분기되는 도로",20100205,None,None,20100205,20220506105800,LineString,"[[127.48761224689987, 36.636329486126925], [12...","LINESTRING (127.48761 36.63633, 127.48760 36.6..."
3,Feature,43111,21898,상당로143번길,4508303,Sangdang-ro 143beon-gil,20100205,3,4,0,북문로2가 140,북문로2가 66,11.0,330.0,10,"상당로의 시작지점에서부터 약 1,430m지점에서 왼쪽으로 분기되는 도로",20100205,90,"광역도로 정비 및 부여사유, 부여방식 정비",20141030,20141030224133,LineString,"[[127.48989140128728, 36.64102936454226], [127...","LINESTRING (127.48989 36.64103, 127.48976 36.6..."
4,Feature,43111,22105,상당로69번길,4508333,Sangdang-ro 69beon-gil,20100205,3,4,0,서문동 224-6,서문동 224-6,12.0,383.0,10,상당로의 시작지점에서부터 약 690m지점에서 왼쪽으로 분기되는 도로,20100205,90,"광역도로 정비 및 부여사유, 부여방식 정비",20141030,20141030224133,LineString,"[[127.49062063280654, 36.634473781482676], [12...","LINESTRING (127.49062 36.63447, 127.48997 36.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Feature,43111,23025,상당로59번길,4508331,Sangdang-ro 59beon-gil,20100205,3,4,1,서문동 224-6,서문동 224-6,4.0,38.0,10,상당로의 시작지점에서부터 약 590m지점에서 왼쪽으로 분기되는 도로,20100205,90,"광역도로 정비 및 부여사유, 부여방식 정비",20141030,20141030224133,LineString,"[[127.48852227274604, 36.6336349713935], [127....","LINESTRING (127.48852 36.63363, 127.48853 36.6..."
134,Feature,43111,23625,사직대로361번길,4508270,Sajik-daero 361beon-gil,20100205,3,4,1,북문로2가 66,영동 91,4.0,43.0,10,"사직대로의 시작지점에서부터 약 3,610m지점에서 왼쪽으로 분기되는 도로",20100205,99,직권수정(속성변경),20141030,20160818102419,LineString,"[[127.48743204282678, 36.638166592426494], [12...","LINESTRING (127.48743 36.63817, 127.48737 36.6..."
135,Feature,43111,23017,사직대로361번길,4508270,Sajik-daero 361beon-gil,20100205,3,4,1,북문로2가 66,영동 91,12.0,66.0,10,"사직대로의 시작지점에서부터 약 3,610m지점에서 왼쪽으로 분기되는 도로",20100205,99,직권수정(속성변경),20141030,20160818102419,LineString,"[[127.48687008838746, 36.643992835460075], [12...","LINESTRING (127.48687 36.64399, 127.48693 36.6..."
136,Feature,43111,23099,성안로,3236031,Seongan-ro,20100205,3,3,1,북문로1가 210-1,북문로1가 210-1,4.0,67.0,10,"옛 청주성 안의 도로를 의미하며 현재의 성안길 명칭 및 역사성, 지역특성 반영",20100205,99,직권수정(속성변경),20141030,20170801153930,LineString,"[[127.48911502318998, 36.635066684364496], [12...","LINESTRING (127.48912 36.63507, 127.48913 36.6..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_roadname_road.iterrows():
    popup_text = f"도로명:{row['properties.RN']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 5, 'opacity': 0.7}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 인도(보도) 시각화

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/17.청주시_인도(보도).geojson') as geojson_file:
    geojson_data = json.load(geojson_file)
sidewalk_df = pd.json_normalize(geojson_data['features'])
sidewalk_df['geometry'] = sidewalk_df['geometry.coordinates'].apply(make_lin)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
sidewalk_df = gpd.GeoDataFrame(sidewalk_df, geometry='geometry')

In [ ]:
sidewalk_df

,type,properties.UFID,properties.QUAL,properties.BYYN,properties.KIND,geometry.type,geometry.coordinates,geometry
0,Feature,100036706005A00317ac9293140644fe9,미분류,무,인도,LineString,"[[127.366950583596, 36.73375193848906], [127.3...","LINESTRING (127.36695 36.73375, 127.36710 36.7..."
1,Feature,100036706017A003158212dfea9cb45a3,미분류,무,인도,LineString,"[[127.4207549271256, 36.70643886461052], [127....","LINESTRING (127.42075 36.70644, 127.42081 36.7..."
2,Feature,100036706019A00316897f36b559b40cb,미분류,무,인도,LineString,"[[127.46555847387654, 36.72275754461322], [127...","LINESTRING (127.46556 36.72276, 127.46623 36.7..."
3,Feature,100036706005A0031b8505464b1c24859,미분류,무,인도,LineString,"[[127.37329220144028, 36.73382215731717], [127...","LINESTRING (127.37329 36.73382, 127.37344 36.7..."
4,Feature,100036707013A003144a5544af6434dde,미분류,무,인도,LineString,"[[127.55719890174112, 36.708660601738984], [12...","LINESTRING (127.55720 36.70866, 127.55719 36.7..."
...,...,...,...,...,...,...,...,...
8291,Feature,100036707061A00316d0d86e43e494762,미분류,유,자전거도로,LineString,"[[127.50091747058062, 36.58492855661412], [127...","LINESTRING (127.50092 36.58493, 127.50090 36.5..."
8292,Feature,100036707051A003117ece3163c21405f,미분류,유,자전거도로,LineString,"[[127.50047534890629, 36.606728352230185], [12...","LINESTRING (127.50048 36.60673, 127.50038 36.6..."
8293,Feature,100036707061A00316c3bc53923a84061,블록,유,자전거도로,LineString,"[[127.49900780350454, 36.59626736081363], [127...","LINESTRING (127.49901 36.59627, 127.49936 36.5..."
8294,Feature,100036707051A003109f484c9f78248a2,아스콘,유,자전거도로,LineString,"[[127.4995607108569, 36.61480389290932], [127....","LINESTRING (127.49956 36.61480, 127.49956 36.6..."


In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_sidewalk = sidewalk_df[sidewalk_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_sidewalk

,type,properties.UFID,properties.QUAL,properties.BYYN,properties.KIND,geometry.type,geometry.coordinates,geometry
0,Feature,100036706050A003189376b14e5964828,미분류,무,인도,LineString,"[[127.49062338110333, 36.6417421675925], [127....","LINESTRING (127.49062 36.64174, 127.49064 36.6..."
1,Feature,100036706050A003101d5f9fbadf84488,미분류,무,인도,LineString,"[[127.48804192410691, 36.63751484423243], [127...","LINESTRING (127.48804 36.63751, 127.48806 36.6..."
2,Feature,100036706050A00314c2a46c79bfa4ef8,미분류,무,인도,LineString,"[[127.4908535706583, 36.6425890088251], [127.4...","LINESTRING (127.49085 36.64259, 127.49079 36.6..."
3,Feature,100036706050A003158cc0af359c945e6,미분류,무,인도,LineString,"[[127.48891283162526, 36.6390439338725], [127....","LINESTRING (127.48891 36.63904, 127.48889 36.6..."
4,Feature,100036706050A00315cf42551cbd149da,미분류,무,인도,LineString,"[[127.48642173123054, 36.63556500043981], [127...","LINESTRING (127.48642 36.63557, 127.48645 36.6..."
...,...,...,...,...,...,...,...,...
116,Feature,100036706050A00316998a65ea4ad440f,아스콘,유,자전거도로,LineString,"[[127.49112468574835, 36.631374516942394], [12...","LINESTRING (127.49112 36.63137, 127.49110 36.6..."
117,Feature,100036706050A0031db4c79d55cb84d39,블록,유,자전거도로,LineString,"[[127.49075989252123, 36.63456720470323], [127...","LINESTRING (127.49076 36.63457, 127.49073 36.6..."
118,Feature,100036706050A0031dc05c4cbc88a4fe9,아스콘,유,자전거도로,LineString,"[[127.491361592062, 36.62943927534968], [127.4...","LINESTRING (127.49136 36.62944, 127.49132 36.6..."
119,Feature,100036706050A0031881c48839a5441fb,블록,유,자전거도로,LineString,"[[127.49060087641541, 36.63567678716627], [127...","LINESTRING (127.49060 36.63568, 127.49049 36.6..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_sidewalk.iterrows():
    popup_text = f"도로명:{row['properties.KIND']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 상권정보 시각화

In [ ]:
trading_area_df = pd.read_csv('SBJ_2309_001/18.청주시_상권정보.csv')
trading_area_df = geo_transform(trading_area_df)
trading_area_df

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,geometry
0,R,학문/교육,R04,학원-어학,R04A01,학원-외국어/어학,P85502,외국어학원,4311259000,성화.개신.죽림동,"충청북도 청주시 서원구 복대로 7, (개신동)",127.446061,36.617928,POINT (127.44606 36.61793)
1,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,4311154500,성안동,"충청북도 청주시 상당구 대성로 38, (서운동)",127.493371,36.628602,POINT (127.49337 36.62860)
2,D,소매,D07,가정/주방/인테리어,D07A15,유리/페인트/철물건축자재,G47519,"페인트, 유리 및 기타 건설자재 소매업",4311257000,수곡1동,"충청북도 청주시 서원구 매봉로 64, (수곡동, 산남주공3차)",127.478662,36.616083,POINT (127.47866 36.61608)
3,D,소매,D15,가구소매,D15A01,일반가구소매,G47520,가구 소매업,4311231000,남이면,"충청북도 청주시 서원구 남이면 청남로 1272, (척산리)",127.440092,36.562169,POINT (127.44009 36.56217)
4,R,학문/교육,R04,학원-어학,R04A01,학원-외국어/어학,P85502,외국어학원,4311454000,율량.사천동,"충청북도 청주시 청원구 공항로 118, (율량동)",127.485764,36.667772,POINT (127.48576 36.66777)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46118,L,부동산,L01,부동산중개,L01A01,부동산중개,L68221,부동산 자문 및 중개업,4311454000,율량.사천동,"충청북도 청주시 청원구 사뜸로61번길 106, (사천동, 대창5차)",127.476366,36.664730,POINT (127.47637 36.66473)
46119,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311425300,오창읍,"충청북도 청주시 청원구 오창읍 두릉유리로 1130-15, (장대리)",127.455312,36.741021,POINT (127.45531 36.74102)
46120,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311154500,성안동,"충청북도 청주시 상당구 무심동로390번길 15, (서문동)",127.484757,36.634341,POINT (127.48476 36.63434)
46121,D,소매,D02,선물/팬시/기념품,D02A02,꽃집/꽃배달,G47851,화초 및 산식물 소매업,4311374700,가경동,"충청북도 청주시 흥덕구 풍년로198번길 56-1, (가경동)",127.432312,36.631408,POINT (127.43231 36.63141)


['음식', '소매', '생활서비스', '학문/교육', '부동산', '관광/여가/오락', '숙박', '스포츠'],

In [ ]:
# a = filtered_trading_area[filtered_trading_area['com_lc_nm'] == '소매']
# a['com_mc_nm'].unique()

NameError: name 'filtered_trading_area' is not defined

In [ ]:
# filtered_trading_area['com_lc_nm'].value_counts()

com_lc_nm
소매          1629
음식           869
생활서비스        470
학문/교육         84
관광/여가/오락      49
숙박            36
부동산           34
스포츠            7
Name: count, dtype: int64

In [ ]:
# a['com_mc_nm'].value_counts()

com_mc_nm
의복의류             568
종합소매점            187
가방/신발/액세서리       162
음/식료품소매          118
화장품소매            101
가정/주방/인테리어        83
사진/광학/정밀기기소매      70
시계/귀금속소매          52
선물/팬시/기념품         41
건강/미용식품           38
가구소매              31
사무/문구/컴퓨터         28
의약/의료품소매          21
기타판매업             21
운동/경기용품소매         20
철물/난방/건설자재소매      19
책/서적/도서           16
가전제품소매            16
유아용품               7
자동차/자동차용품          6
애견/애완/동물           6
취미/오락관련소매          6
중고품소매/교환           5
예술품/골동품/수석/분재      4
종교용품판매             2
페인트/유리제품소매         1
Name: count, dtype: int64

In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_trading_area = trading_area_df[trading_area_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_trading_area

,com_lc_cd,com_lc_nm,com_mc_cd,com_mc_nm,com_sc_cd,com_sc_nm,ksic_cd,ksic_nm,adb_emd_cd,adb_emd_nm,road_address,lon,lat,geometry
0,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,4311154500,성안동,"충청북도 청주시 상당구 남사로140번길 6, (남문로2가)",127.490409,36.631851,POINT (127.49041 36.63185)
1,D,소매,D10,건강/미용식품,D10A07,건강원,G47216,건강보조식품 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로3번길 18-1, (석교동)",127.490050,36.628551,POINT (127.49005 36.62855)
2,D,소매,D05,의복의류,D05A10,한복/갈옷/민속옷,G47412,한복 소매업,4311154500,성안동,"충청북도 청주시 상당구 무심동로336번길 100-2, (남문로1가)",127.488456,36.631302,POINT (127.48846 36.63130)
3,D,소매,D05,의복의류,D05A02,캐쥬얼/스포츠의류,G47416,셔츠 및 기타 의복 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로 55, (남문로2가)",127.490343,36.633284,POINT (127.49034 36.63328)
4,D,소매,D07,가정/주방/인테리어,D07A02,주방용품,G47592,식탁 및 주방용품 소매업,4311154500,성안동,"충청북도 청주시 상당구 상당로1번길 22, (석교동)",127.489694,36.628361,POINT (127.48969 36.62836)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173,D,소매,D23,자동차/자동차용품,D23A11,자동차유리전문점,G45219,기타 자동차신품 부품 및 내장품 판매업,4311152500,중앙동,"충청북도 청주시 상당구 사북로 138, (영동)",127.486152,36.644819,POINT (127.48615 36.64482)
3174,F,생활서비스,F05,인력/고용/용역알선,F05A02,인력공급/고용알선,NaN,NaN,4311154500,성안동,"충청북도 청주시 상당구 용담로 7, (문화동, 문화동센트럴칸타빌)",127.492115,36.632598,POINT (127.49211 36.63260)
3175,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,한식 음식점업,4311154500,성안동,"충청북도 청주시 상당구 사직대로362번길 46, (서문동)",127.487471,36.634714,POINT (127.48747 36.63471)
3176,D,소매,D16,화장품소매,D16A01,화장품판매점,G47813,화장품 및 방향제 소매업,4311152500,중앙동,"충청북도 청주시 상당구 상당로 127, (북문로2가)",127.489698,36.639536,POINT (127.48970 36.63954)


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_trading_area.iterrows():
    popup_text = f"{row['com_lc_nm']}-{row['com_mc_nm']}-{row['com_sc_nm']}"
    marker = folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        fill=True,
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

### 청주시 주민등록인구 현황

In [ ]:
pop_df = pd.read_csv('SBJ_2309_001/22.청주시_주민등록인구현황.csv')
pop_df

,adm_cd,adm_nm,year,m_pop,fm_pop
0,4311131000,충청북도 청주시 상당구 낭성면,201701,1232,1140
1,4311132000,충청북도 청주시 상당구 미원면,201701,2757,2636
2,4311133000,충청북도 청주시 상당구 가덕면,201701,2259,2066
3,4311134000,충청북도 청주시 상당구 남일면,201701,3901,3746
4,4311135000,충청북도 청주시 상당구 문의면,201701,2383,2165
...,...,...,...,...,...
3091,4311451000,충청북도 청주시 청원구 우암동,202212,6209,6073
3092,4311452000,충청북도 청주시 청원구 내덕1동,202212,4197,4124
3093,4311453000,충청북도 청주시 청원구 내덕2동,202212,6823,6453
3094,4311454000,충청북도 청주시 청원구 율량.사천동,202212,24328,24478


In [ ]:
pop_df['adm_nm'].unique()

array(['충청북도 청주시 상당구 낭성면', '충청북도 청주시 상당구 미원면', '충청북도 청주시 상당구 가덕면',
       '충청북도 청주시 상당구 남일면', '충청북도 청주시 상당구 문의면', '충청북도 청주시 상당구 중앙동',
       '충청북도 청주시 상당구 성안동', '충청북도 청주시 상당구 탑대성동', '충청북도 청주시 상당구 영운동',
       '충청북도 청주시 상당구 금천동', '충청북도 청주시 상당구 용담.명암.산성동', '충청북도 청주시 상당구 용암1동',
       '충청북도 청주시 상당구 용암2동', '충청북도 청주시 서원구 남이면', '충청북도 청주시 서원구 현도면',
       '충청북도 청주시 서원구 사직1동', '충청북도 청주시 서원구 사직2동', '충청북도 청주시 서원구 사창동',
       '충청북도 청주시 서원구 모충동', '충청북도 청주시 서원구 산남동', '충청북도 청주시 서원구 분평동',
       '충청북도 청주시 서원구 수곡1동', '충청북도 청주시 서원구 수곡2동', '충청북도 청주시 서원구 성화.개신.죽림동',
       '충청북도 청주시 흥덕구 오송읍', '충청북도 청주시 흥덕구 강내면', '충청북도 청주시 흥덕구 옥산면',
       '충청북도 청주시 흥덕구 운천.신봉동', '충청북도 청주시 흥덕구 복대1동', '충청북도 청주시 흥덕구 복대2동',
       '충청북도 청주시 흥덕구 가경동', '충청북도 청주시 흥덕구 봉명1동', '충청북도 청주시 흥덕구 봉명2.송정동',
       '충청북도 청주시 흥덕구 강서제1동', '충청북도 청주시 흥덕구 강서제2동', '충청북도 청주시 청원구 내수읍',
       '충청북도 청주시 청원구 오창읍', '충청북도 청주시 청원구 북이면', '충청북도 청주시 청원구 우암동',
       '충청북도 청주시 청원구 내덕1동', '충청북도 청주시 청원구 내덕2동', '충청북도 청주시 청원구 율량.사천동',
       '충청북도 청주시 청원구 오근장

### 청주시 건물 노후도

구조별 수명에 대해 노후도 및 위험도 책정

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/23.청주시_건물노후도.geojson', 'r') as geojson_file:
    geojson_data = json.load(geojson_file)
building_year_df = pd.json_normalize(geojson_data['features'])
building_year_df = building_year_df.dropna(subset=['properties.old_year'])
building_year_df['properties.old_year'] = building_year_df['properties.old_year'].astype(int)
building_year_df['geometry'] = building_year_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
building_year_df = gpd.GeoDataFrame(building_year_df, geometry='geometry')

In [ ]:
building_year_df

,type,properties.pnu,properties.emd_nm,properties.bld_nm,properties.dong_nm,properties.gfa,properties.strct_nm,properties.usage_nm,properties.useapr_day,properties.old_year,geometry.type,geometry.coordinates,geometry
0,Feature,4311110100100020001,충청북도 청주시 상당구영동,None,None,145.26,벽돌구조,단독주택,1978-01-20,46,MultiPolygon,"[[[[127.48671936764237, 36.644922806739345], [...","POLYGON ((127.48672 36.64492, 127.48673 36.644..."
1,Feature,4311110100100020002,충청북도 청주시 상당구영동,None,None,92.43,경량철골구조,단독주택,2011-04-12,13,MultiPolygon,"[[[[127.48644934729707, 36.644848446412425], [...","POLYGON ((127.48645 36.64485, 127.48657 36.644..."
2,Feature,4311110100100020003,충청북도 청주시 상당구영동,None,주2,173.13,경량철골구조,제2종근린생활시설,2011-09-14,13,MultiPolygon,"[[[[127.48626989607264, 36.644893917411586], [...","POLYGON ((127.48627 36.64489, 127.48641 36.644..."
3,Feature,4311110100100020004,충청북도 청주시 상당구영동,None,None,140.13,벽돌구조,제1종근린생활시설,1978-01-16,46,MultiPolygon,"[[[[127.48673982638863, 36.64474369781375], [1...","POLYGON ((127.48674 36.64474, 127.48667 36.644..."
5,Feature,4311110100100040001,충청북도 청주시 상당구영동,None,None,84.60,경량철골구조,제2종근린생활시설,2001-05-11,23,MultiPolygon,"[[[[127.48620407621114, 36.64486541245635], [1...","POLYGON ((127.48620 36.64487, 127.48620 36.644..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83476,Feature,4311431046102950000,충청북도 청주시 청원구 북이면 송정리,None,주건축물제1동,942.72,경량철골구조,동.식물 관련시설,2011-07-06,13,MultiPolygon,"[[[[127.56574298112129, 36.76203021541931], [1...","POLYGON ((127.56574 36.76203, 127.56570 36.761..."
83477,Feature,4311431046103110000,충청북도 청주시 청원구 북이면 송정리,None,None,786.03,경량철골구조,동.식물 관련시설,2015-12-07,9,MultiPolygon,"[[[[127.56793701208925, 36.76389802405521], [1...","POLYGON ((127.56794 36.76390, 127.56817 36.763..."
83478,Feature,4311431046103350004,충청북도 청주시 청원구 북이면 송정리,None,None,85.05,경량철골구조,제2종근린생활시설,1997-01-10,27,MultiPolygon,"[[[[127.57013584496227, 36.76047315387015], [1...","POLYGON ((127.57014 36.76047, 127.57025 36.760..."
83479,Feature,4311431046103390001,충청북도 청주시 청원구 북이면 송정리,None,None,195.00,일반철골구조,제1종근린생활시설,2010-11-24,14,MultiPolygon,"[[[[127.56992785024588, 36.761865976768725], [...","POLYGON ((127.56993 36.76187, 127.56991 36.761..."


In [ ]:
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_building_year = building_year_df[building_year_df['geometry'].within(polygon)].reset_index(drop=True)
filtered_building_year

,type,properties.pnu,properties.emd_nm,properties.bld_nm,properties.dong_nm,properties.gfa,properties.strct_nm,properties.usage_nm,properties.useapr_day,properties.old_year,geometry.type,geometry.coordinates,geometry
0,Feature,4311110100100020001,충청북도 청주시 상당구영동,None,None,145.26,벽돌구조,단독주택,1978-01-20,46,MultiPolygon,"[[[[127.48671936764237, 36.644922806739345], [...","POLYGON ((127.48672 36.64492, 127.48673 36.644..."
1,Feature,4311110100100020002,충청북도 청주시 상당구영동,None,None,92.43,경량철골구조,단독주택,2011-04-12,13,MultiPolygon,"[[[[127.48644934729707, 36.644848446412425], [...","POLYGON ((127.48645 36.64485, 127.48657 36.644..."
2,Feature,4311110100100020003,충청북도 청주시 상당구영동,None,주2,173.13,경량철골구조,제2종근린생활시설,2011-09-14,13,MultiPolygon,"[[[[127.48626989607264, 36.644893917411586], [...","POLYGON ((127.48627 36.64489, 127.48641 36.644..."
3,Feature,4311110100100020004,충청북도 청주시 상당구영동,None,None,140.13,벽돌구조,제1종근린생활시설,1978-01-16,46,MultiPolygon,"[[[[127.48673982638863, 36.64474369781375], [1...","POLYGON ((127.48674 36.64474, 127.48667 36.644..."
4,Feature,4311110100100040001,충청북도 청주시 상당구영동,None,None,84.60,경량철골구조,제2종근린생활시설,2001-05-11,23,MultiPolygon,"[[[[127.48620407621114, 36.64486541245635], [1...","POLYGON ((127.48620 36.64487, 127.48620 36.644..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143,Feature,4311111200104120017,충청북도 청주시 상당구 수동,None,None,199.83,벽돌구조,단독주택,1990-10-18,34,MultiPolygon,"[[[[127.49108479582297, 36.644601827356276], [...","POLYGON ((127.49108 36.64460, 127.49093 36.644..."
2144,Feature,4311111200104120020,충청북도 청주시 상당구 수동,None,None,162.38,철근콘크리트구조,단독주택,2015-11-17,9,MultiPolygon,"[[[[127.49072487086501, 36.64459902566444], [1...","POLYGON ((127.49072 36.64460, 127.49077 36.644..."
2145,Feature,4311111200104120027,충청북도 청주시 상당구 수동,None,None,118.11,벽돌구조,단독주택,1983-12-08,41,MultiPolygon,"[[[[127.49048536397038, 36.64466430392876], [1...","POLYGON ((127.49049 36.64466, 127.49048 36.644..."
2146,Feature,4311111200104120027,충청북도 청주시 상당구 수동,None,None,99.94,벽돌구조,단독주택,1985-12-28,39,MultiPolygon,"[[[[127.490738466091, 36.644671104512796], [12...","POLYGON ((127.49074 36.64467, 127.49074 36.644..."


In [ ]:
# 지도의 중심 좌표 설정
m = folium.Map(location=[36.627797, 127.511943], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in filtered_building_year.iterrows():
    popup_text = f"구조명:{row['properties.strct_nm']}, 용도명:{row['properties.usage_nm']}<br>연식:{row['properties.old_year']}년"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))
    
m

## turtle로 그림그리는 방안 생각해보기?